Основное


- Дедлайн - 8 мая 23:59
- Целевая метрика precision@5
- Бейзлайн решения - MainRecommender
- Сдаем ссылку на github с решением. В решении должны быть отчетливо видна метрика на новом тестовом сете из файла retail_test1.csv, то есть вам нужно для всех юзеров из этого файла выдать выши рекомендации, и посчитать на actual покупках precision@5.


!! Мы не рассматриваем холодный старт для пользователя, все наши пользователя одинаковы во всех сетах, поэтому нужно позаботиться об их исключении из теста.
Hints:
Сначала просто попробуйте разные параметры MainRecommender:
- N в топ-N товарах при формировании user-item матирцы (сейчас топ-5000)
- Различные веса в user-item матрице (0/1, кол-во покупок, log(кол-во покупок + 1), сумма покупки, ...)
- Разные взвешивания матрицы (TF-IDF, BM25 - у него есть параметры)
- Разные смешивания рекомендаций (обратите внимание на бейзлайн - прошлые покупки юзера)

Сделайте MVP - минимально рабочий продукт - (пусть даже top-popular), а потом его улучшайте
Если вы делаете двухуровневую модель - следите за валидацией

## Import libs

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit import als

# Модель второго уровня
from lightgbm import LGBMClassifier

import os, sys
module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

# Написанные нами функции
from src.metrics import precision_at_k, recall_at_k
from src.utils import prefilter_items
from src.recommenders import MainRecommender

from catboost import CatBoostClassifier, Pool
import warnings
warnings.simplefilter('ignore')
warnings.filterwarnings('ignore')

## Other Functions

In [4]:
N_PREDICT = 50 
def make_df_result(df):
    df_result = df.groupby(USER_COL)[ITEM_COL].unique().reset_index()
    df_result.columns=[USER_COL, ACTUAL_COL]
    return df_result
# функция для получения рекомендаций
def make_df_recs(df_result, names_rec, list_models, N=N_PREDICT):
    for i in range(len(names_rec)):
        recommend_model = list_models[i]
        df_result[names_rec[i]] = df_result[USER_COL].apply(lambda x: recommend_model(x, N))
def calc_precision(df_data, top_k):
    for col_name in df_data.columns[2:]:
        yield col_name, df_data.apply(lambda row: precision_at_k(row[col_name], row[ACTUAL_COL], k=top_k), axis=1).mean()
def calc_recall(df_data, top_k):
    for col_name in df_data.columns[2:]:
        yield col_name, df_data.apply(lambda row: recall_at_k(row[col_name], row[ACTUAL_COL], k=top_k), axis=1).mean()
def rerank(user_id):
    return df_ranker_predict[df_ranker_predict[USER_COL]==user_id].sort_values('proba_item_purchase', ascending=False).head(5).item_id.tolist()

In [5]:
def make_3_df(data, VAL_MATCHER_WEEKS, VAL_RANKER_WEEKS):
    data_train_matcher = data[data['week_no'] < data['week_no'].max() - (VAL_MATCHER_WEEKS + VAL_RANKER_WEEKS)]
    data_val_matcher = data[(data['week_no'] >= data['week_no'].max() - (VAL_MATCHER_WEEKS + VAL_RANKER_WEEKS)) &
                      (data['week_no'] < data['week_no'].max() - (VAL_RANKER_WEEKS))]
    data_train_ranker = data_val_matcher.copy()  
    data_val_ranker = data[data['week_no'] >= data['week_no'].max() - VAL_RANKER_WEEKS]
    return data_train_matcher, data_val_matcher, data_train_ranker, data_val_ranker

In [6]:
def find_comm_us(data_train_matcher, data_val_matcher, data_train_ranker, data_val_ranker, df_test):
    # ищем общих пользователей
    common_users = list(set(data_train_matcher.user_id.values)&(set(data_val_matcher.user_id.values))&(set(data_val_ranker.user_id.values))&(set(df_test.user_id.values)))

    # оставляем общих пользователей
    data_train_matcher = data_train_matcher[data_train_matcher.user_id.isin(common_users)]
    data_val_matcher = data_val_matcher[data_val_matcher.user_id.isin(common_users)]
    data_train_ranker = data_train_ranker[data_train_ranker.user_id.isin(common_users)]
    data_val_ranker = data_val_ranker[data_val_ranker.user_id.isin(common_users)]
    df_test = df_test[df_test.user_id.isin(common_users)]
    return data_train_matcher, data_val_matcher, data_train_ranker, data_val_ranker, df_test

## Read data

In [9]:
data = pd.read_csv('../webinar_2/webinar_2/data/retail_train.csv')
item_features = pd.read_csv('../webinar_2/webinar_2/data/product.csv')
user_features = pd.read_csv('../webinar_2/webinar_2/data/hh_demographic.csv')
df_test = pd.read_csv('retail_test1.csv')

In [607]:
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


# Set global const

In [3]:
ITEM_COL = 'item_id'
USER_COL = 'user_id'
ACTUAL_COL = 'actual'

# N = Neighbors
N_PREDICT = 50 

## Process features dataset

In [11]:
# column processing
item_features.columns = [col.lower() for col in item_features.columns]
user_features.columns = [col.lower() for col in user_features.columns]

item_features.rename(columns={'product_id': ITEM_COL}, inplace=True)
user_features.rename(columns={'household_key': USER_COL }, inplace=True)

In [12]:
user_features['user_id'].nunique()

801

In [13]:
data['week_no'].nunique()

95

# Split dataset for train, eval, test

In [577]:
# Важна схема обучения и валидации!
# -- давние покупки -- | .-- ? недель -- | -- ? недель -- 
# подобрать размер 2-ого датасета (? недель) --> learning curve (зависимость метрики recall@k от размера датасета)
VAL_MATCHER_WEEKS = 8
VAL_RANKER_WEEKS = 1
data_train_matcher, data_val_matcher, data_train_ranker, data_val_ranker = make_3_df(data, VAL_MATCHER_WEEKS, VAL_RANKER_WEEKS)


In [578]:
n_items_before = data_train_matcher['item_id'].nunique()
data_train_matcher = prefilter_items(data_train_matcher, item_features=item_features, take_n_popular=4000)
n_items_after = data_train_matcher['item_id'].nunique()
print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

Decreased # items from 83685 to 4001


# Make cold-start to warm-start

In [579]:
# ищем и остовляем общих пользователей
data_train_matcher, data_val_matcher, data_train_ranker, data_val_ranker, df_test = find_comm_us(data_train_matcher, data_val_matcher, data_train_ranker, data_val_ranker, df_test)
            

# Init/train recommender

In [580]:
recommender = MainRecommender(data_train_matcher)

## Eval recall of matching

In [582]:
# подготовим датафрейм с данными о фактических покупках
result_eval_matcher = make_df_result(data_val_matcher)
#result_eval_matcher.head(2)
names_rec = ['own_rec']
list_models = [recommender.get_own_recommendations ]
make_df_recs(result_eval_matcher, names_rec, list_models, N=N_PREDICT)
#result_eval_matcher.head(2)

In [583]:
#смотрим precision@5
sorted(calc_precision(result_eval_matcher, 5), key=lambda x: x[1],reverse=True)

[('own_rec', 0.258705035971223)]

In [584]:
#смотрим recall@5
sorted(calc_recall(result_eval_matcher, 5), key=lambda x: x[1],reverse=True)

[('own_rec', 0.015704528590766296)]

In [585]:
# подготовим датафрейм с данными о фактических покупках
result_test = make_df_result(df_test)
#result_test.head(2)
names_rec = ['own_rec' ]
list_models = [recommender.get_own_recommendations]

make_df_recs(result_test, names_rec, list_models, N=N_PREDICT)
#смотрим precision@5
sorted(calc_precision(result_test, 5), key=lambda x: x[1],reverse=True)

[('own_rec', 0.14517985611510792)]

In [586]:
#смотрим recall@5
sorted(calc_recall(result_test, 5), key=lambda x: x[1],reverse=True)

[('own_rec', 0.019192918234939283)]

# Ranking part 

In [587]:
# взяли пользователей из трейна для ранжирования
df_match_candidates = pd.DataFrame(data_train_ranker[USER_COL].unique())
df_match_candidates.columns = [USER_COL]

In [588]:
# собираем кандитатов с первого этапа (matcher)
df_match_candidates['candidates'] = df_match_candidates[USER_COL].apply(lambda x: recommender.get_own_recommendations(x, N=N_PREDICT))

In [589]:
names_rec = ['sim_item_rec', 'als_rec','sim_user_rec']
list_models = [recommender.get_similar_items_recommendation, recommender.get_als_recommendations, 
               recommender.get_similar_users_recommendation]

make_df_recs(df_match_candidates, names_rec, list_models, N=N_PREDICT)

In [590]:
#собираем кандидатов объединением разных рекоммендаций и добавляем 1 из топ
df_match_candidates['candidates'] = df_match_candidates['user_id'].apply(lambda x: list(df_match_candidates[df_match_candidates['user_id']==x]['candidates'])[0]+
                                                                        list(df_match_candidates[df_match_candidates['user_id']==x]['sim_item_rec'])[0]+
                                                                        list(df_match_candidates[df_match_candidates['user_id']==x]['als_rec'])[0]+
                                                                        list(list(result_eval_matcher[result_eval_matcher['user_id']==x]['actual'])[0])[:1]+
                                                                        list(df_match_candidates[df_match_candidates['user_id']==x]['sim_user_rec'])[0]
                                                                        )
# разворачиваем товары
df_items = df_match_candidates.apply(lambda x: pd.Series(x['candidates']), axis=1).stack().reset_index(level=1, drop=True)
df_items.name = 'item_id'
df_match_candidates = df_match_candidates.drop(['candidates', 'sim_item_rec','als_rec', 'sim_user_rec'], axis=1).join(df_items)
#df_match_candidates.head(3)
df_ranker_train = data_train_ranker[[USER_COL, ITEM_COL]].copy()
df_ranker_train['target'] = 1  # тут только покупки
df_ranker_train = df_match_candidates.merge(df_ranker_train, on=[USER_COL, ITEM_COL], how='left')
df_ranker_train['target'].fillna(0, inplace= True)

# Новые фичи

In [591]:
df_join_train_matcher = pd.concat([data_train_matcher, data_val_matcher])

## Подготавливаем фичи для обучения модели

In [592]:
df_ranker_train = df_ranker_train.merge(item_features, on='item_id', how='left')
df_ranker_train = df_ranker_train.merge(user_features, on='user_id', how='left')

df_ranker_train.head(2)

,user_id,item_id,target,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc
0,2070,1105426,0.0,69.0,DELI,Private,SANDWICHES,SANDWICHES - (COLD),,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown
1,2070,879194,0.0,69.0,DRUG GM,Private,DIAPERS & DISPOSABLES,BABY DIAPERS,14 CT,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown


### Поведенческие фичи

##### Чтобы считать поведенческие фичи, нужно учесть все данные что были до data_val_ranker

In [593]:
#сумма продаж по item
df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=ITEM_COL).agg('sales_value').sum().rename('total_item_sales_value'), how='left',on=ITEM_COL)
#суммарный объем по item
df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=ITEM_COL).agg('quantity').sum().rename('total_quantity_value'), how='left',on=ITEM_COL)
#количество юзеров, которые купили item
df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=ITEM_COL).agg(USER_COL).count().rename('item_freq'), how='left',on=ITEM_COL)
#частота появления юзера
df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=USER_COL).agg(USER_COL).count().rename('user_freq'), how='left',on=USER_COL)
#суммарные продажи по юзеру
df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=USER_COL).agg('sales_value').sum().rename('total_user_sales_value'), how='left',on=USER_COL)
#количество проданного товара в неделю
df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=ITEM_COL).agg('quantity').sum().rename('item_quantity_per_week')/df_join_train_matcher.week_no.nunique(), how='left',on=ITEM_COL)
#количество проданных товаров по юзеру в неделю
df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=USER_COL).agg('quantity').sum().rename('user_quantity_per_week')/df_join_train_matcher.week_no.nunique(), how='left',on=USER_COL)

#среднее количество товара в корзине
df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=ITEM_COL).agg('quantity').sum().rename('item_quantity_per_basket')/df_join_train_matcher.basket_id.nunique(), how='left',on=ITEM_COL)
#средний объем покупок в корзине по юзеру
df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=USER_COL).agg('quantity').sum().rename('user_quantity_per_baskter')/df_join_train_matcher.basket_id.nunique(), how='left',on=USER_COL)

#частота появления товара в корзинах
df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=ITEM_COL).agg(USER_COL).count().rename('item_freq_per_basket')/df_join_train_matcher.basket_id.nunique(), how='left',on=ITEM_COL)
#частота корзин у юзера
df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=USER_COL).agg(USER_COL).count().rename('user_freq_per_basket')/df_join_train_matcher.basket_id.nunique(), how='left',on=USER_COL)
# количество корзин у пользователя
df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=USER_COL).agg('basket_id').nunique().rename('count_basket'), how='left',on=USER_COL)


Фичи user_id: - Средний чек - Средняя сумма покупки 1 товара в каждой категории - Кол-во покупок в каждой категории - Частотность покупок раз/месяц - Долю покупок в выходные - Долю покупок утром/днем/вечером


Фичи item_id: - Кол-во покупок в неделю - Среднее кол-во покупок 1 товара в категории в неделю - (Кол-во покупок в неделю) / (Среднее ол-во покупок 1 товара в категории в неделю) - Цена (Можно посчитать из retil_train.csv) - Цена / Средняя цена товара в категории


Фичи пары user_id - item_id - (Средняя сумма покупки 1 товара в каждой категории (берем категорию item_id)) - (Цена item_id) - (Кол-во покупок юзером конкретной категории в неделю) - (Среднее кол-во покупок всеми юзерами конкретной категории в неделю) - (Кол-во покупок юзером конкретной категории в неделю) / (Среднее кол-во покупок всеми юзерами конкретной категории в неделю)

In [594]:
df_ranker_train['manufacturer'] = df_ranker_train['manufacturer'].astype(str)
df_ranker_train['target'] = df_ranker_train['target'].astype(int)
df_ranker_train['department'].fillna('nan', inplace= True)
df_ranker_train['brand'].fillna('nan', inplace= True)
df_ranker_train['commodity_desc'].fillna('nan', inplace= True)
df_ranker_train['sub_commodity_desc'].fillna('nan', inplace= True)
df_ranker_train['curr_size_of_product'].fillna('nan', inplace= True)
df_ranker_train['age_desc'].fillna('nan', inplace= True)
df_ranker_train['marital_status_code'].fillna('nan', inplace= True)
df_ranker_train['income_desc'].fillna('nan', inplace= True)
df_ranker_train['marital_status_code'].fillna('nan', inplace= True)
df_ranker_train['income_desc'].fillna('nan', inplace= True)
df_ranker_train['homeowner_desc'].fillna('nan', inplace= True)
df_ranker_train['hh_comp_desc'].fillna('nan', inplace= True)
df_ranker_train['household_size_desc'].fillna('nan', inplace= True)
df_ranker_train['kid_category_desc'].fillna('nan', inplace= True)
df_ranker_train['total_quantity_value'].fillna(0, inplace= True)
df_ranker_train['item_freq'].fillna(0, inplace= True)
df_ranker_train['user_freq'].fillna(0, inplace= True)
df_ranker_train['count_basket'].fillna(0, inplace= True)

In [595]:
X_train = df_ranker_train.drop('target', axis=1)
y_train = df_ranker_train[['target']]
cat_feats = ['manufacturer', 'department', 'brand', 'commodity_desc', 'sub_commodity_desc', 'curr_size_of_product',
            'age_desc', 'marital_status_code', 'income_desc', 'homeowner_desc','hh_comp_desc','household_size_desc',
            'kid_category_desc', 'total_quantity_value', 'item_freq', 'user_freq','count_basket']
#cat_feats = X_train.columns[2:].tolist()
X_train[cat_feats] = X_train[cat_feats].astype('category')

#cat_feats
lgb = LGBMClassifier(objective='binary',
                     max_depth=10,
                     n_estimators=500,
                     #learning_rate=0.05,
                     categorical_column=cat_feats)

lgb.fit(X_train, y_train)

train_preds = lgb.predict_proba(X_train)

[LightGBM] [Warning] Accuracy may be bad since you didn't set num_leaves and 2^max_depth > num_leaves


In [596]:
df_ranker_predict = df_ranker_train.copy()
df_ranker_predict['proba_item_purchase'] = train_preds[:,1]
df_ranker_predict.shape
df_ranker_predict = df_ranker_predict.drop_duplicates()
df_ranker_predict.reset_index(drop=True, inplace = True)
df_ranker_predict.shape

(233622, 29)

In [597]:
result_eval_ranker = make_df_result(data_val_ranker)
result_eval_ranker.head(2)

,user_id,actual
0,1,"[856942, 861272, 869465, 877373, 908213, 93391..."
1,6,"[1329768, 897811, 14111230, 822812, 840361, 84..."


In [598]:
names_rec = ['own_rec']
list_models = [recommender.get_own_recommendations]

make_df_recs(result_eval_ranker, names_rec, list_models, N=N_PREDICT)

In [599]:
sorted(calc_precision(result_eval_ranker, 5), key=lambda x: x[1], reverse=True)

[('own_rec', 0.12043165467625899)]

In [600]:
sorted(calc_recall(result_eval_ranker, 50), key=lambda x: x[1], reverse=True)

[('own_rec', 0.06994720805398641)]

In [601]:
result_eval_ranker['reranked_own_rec'] = result_eval_ranker[USER_COL].apply(lambda user_id: rerank(user_id))

In [602]:
print(*sorted(calc_precision(result_eval_ranker, 5), key=lambda x: x[1], reverse=True), sep='\n')

('reranked_own_rec', 0.22302158273381295)
('own_rec', 0.12043165467625899)


In [603]:
result_test = make_df_result(df_test)
result_test.head(2)

,user_id,actual
0,1,"[880007, 883616, 931136, 938004, 940947, 94726..."
1,6,"[956902, 960791, 1037863, 1119051, 1137688, 84..."


In [605]:
result_test['reranked_own_rec'] = result_test[USER_COL].apply(lambda user_id: rerank(user_id))

In [606]:
sorted(calc_precision(result_test, 5), key=lambda x: x[1], reverse=True)

[('reranked_own_rec', 0.263021582733813)]

In [608]:
result_test

,user_id,actual,reranked_own_rec
0,1,"[880007, 883616, 931136, 938004, 940947, 94726...","[853529, 8293439, 9297615, 940947, 856942]"
1,6,"[956902, 960791, 1037863, 1119051, 1137688, 84...","[1024306, 1098844, 1082185, 878996, 862866]"
2,7,"[847270, 855557, 859987, 863407, 895454, 90663...","[836281, 10285022, 1122358, 993838, 1029743]"
3,8,"[846334, 850834, 857503, 862139, 865891, 87829...","[868075, 1029743, 1044078, 5569471, 1074754]"
4,9,"[883404, 995242, 1056005, 889692, 911140, 918046]","[1029743, 962229, 1056005, 883616, 1044078]"
...,...,...,...
1385,2492,"[823153, 835243, 838186, 838882, 845622, 84903...","[883003, 929231, 1106523, 914577, 912004]"
1386,2496,"[829291, 862139, 912704, 933067, 933835, 95537...","[831509, 1056509, 916122, 995876, 865456]"
1387,2497,[6534178],"[1120361, 820291, 1029743, 1103513, 1135834]"
1388,2498,"[1053690, 1076875, 12386123, 858303, 920109, 1...","[901776, 940766, 951197, 865511, 1100379]"


In [609]:
result_test.to_csv('result_test.csv',index=False)

Что смотрела:
1. N в топ-N товарах при формировании user-item матирцы от 500 до 6000
2. Различные веса в user-item матрице ( кол-во покупок, log(кол-во покупок + 1), сумма покупки)
3. Разные взвешивания матрицы (TF-IDF, BM25 - у него есть параметры)
4. Разные смешивания рекомендаций - разное количество разных типов рекомендаций(для own_rec - больше, для остальных меньше, а так же одинаковое количество)
5. По фичам тяжело что то новое было придумать(или то что было в презентации не очень получилось реализовать) - добавила только одну - количество корзин у пользователя
6. пробовала вторую часть делать с помощью catboost-a

Выводы - в том виде, в котором сейчас реализовано получается результат precision@5=0.263
На самом деле, очень много параметров, которые влияют на результат
И сочетаний, которые дают примерно такой же результат (25-27 процентов) достаточно много (сетка и результаты находятся ниже)
Скорее всего на результат сильно повлияют новые фичи во второй части и подход к формированию списка кандидатов в первой части(возможно попроьовать еще другие типы рекоммендаций)
Иногда непонятно на какие метрики смотреть в первой части, тк небольшой recall может дальше дать хороший precision


Поиск лучших параметров

Catboost

In [ ]:
X_train = df_ranker_train.drop('target', axis=1)
y_train = df_ranker_train[['target']]
cat_feats = ['manufacturer', 'department', 'brand', 'commodity_desc', 'sub_commodity_desc', 'curr_size_of_product',
            'age_desc', 'marital_status_code', 'income_desc', 'homeowner_desc','hh_comp_desc','household_size_desc',
            'kid_category_desc', 'total_quantity_value', 'item_freq', 'user_freq','count_basket']


model = CatBoostClassifier(iterations=200,
                           depth=10,
                           learning_rate=0.2,
                           loss_function='Logloss',
                           verbose=True)
model.fit(X_train, y_train, cat_features=cat_feats)
train_preds = model.predict_proba(X_train)

In [2424]:
matcher_weeks = [5, 6, 7, 8, 9, 10, 11, 12]
ranker_weeks = [1, 2, 3, 4, 5, 6]
n_pops = [500, 1000, 2000, 3000, 4000, 5000, 6000]
for VAL_MATCHER_WEEKS in matcher_weeks:
    for VAL_RANKER_WEEKS in ranker_weeks:
        for n_pop in n_pops:
            print(VAL_MATCHER_WEEKS, VAL_RANKER_WEEKS, n_pop)
            data_train_matcher, data_val_matcher, data_train_ranker, data_val_ranker = make_3_df(data, VAL_MATCHER_WEEKS, VAL_RANKER_WEEKS)
            data_train_matcher = prefilter_items(data_train_matcher, item_features=item_features, take_n_popular=n_pop)
            data_train_matcher, data_val_matcher, data_train_ranker, data_val_ranker, df_test = find_comm_us(data_train_matcher, data_val_matcher, data_train_ranker, data_val_ranker, df_test)
            recommender = MainRecommender(data_train_matcher)
                
                # подготовим датафрейм с данными о фактических покупках
            result_eval_matcher = make_df_result(data_val_matcher)
            names_rec = ['own_rec'] #,'sim_item_rec', 'als_rec','sim_user_rec'
            list_models = [recommender.get_own_recommendations] #recommender.get_similar_items_recommendation, recommender.get_als_recommendations, recommender.get_similar_users_recommendation
            make_df_recs(result_eval_matcher, names_rec, list_models, N=N_PREDICT)
            print('train precision@50, recall@50, precision@5, recall@5')
            print(sorted(calc_precision(result_eval_matcher, 50), key=lambda x: x[1],reverse=True))
            print(sorted(calc_recall(result_eval_matcher, 50), key=lambda x: x[1],reverse=True))
            print(sorted(calc_precision(result_eval_matcher, 5), key=lambda x: x[1],reverse=True))
            print(sorted(calc_recall(result_eval_matcher, 5), key=lambda x: x[1],reverse=True))
                
                # подготовим датафрейм с данными о фактических покупках не тесте
            result_test = make_df_result(df_test)
            names_rec = ['own_rec']#,'sim_item_rec', 'als_rec','sim_user_rec'
            list_models = [recommender.get_own_recommendations]#, recommender.get_similar_items_recommendation, recommender.get_als_recommendations, recommender.get_similar_users_recommendation
            make_df_recs(result_test, names_rec, list_models, N=N_PREDICT)
            print('test precision@50, recall@50, precision@5, recall@5')
            print(sorted(calc_precision(result_test, 50), key=lambda x: x[1],reverse=True))
            print(sorted(calc_recall(result_test, 50), key=lambda x: x[1],reverse=True))
            print(sorted(calc_precision(result_test, 5), key=lambda x: x[1],reverse=True))
            print(sorted(calc_recall(result_test, 5), key=lambda x: x[1],reverse=True))

5 1 500



train precision@50, recall@50, precision@5, recall@5
[('own_rec', 0.09594444444444446)]
[('own_rec', 0.07152961951641493)]
[('own_rec', 0.2713888888888889)]
[('own_rec', 0.023017539369174913)]
test precision@50, recall@50, precision@5, recall@5
[('own_rec', 0.06555555555555556)]
[('own_rec', 0.07180706313856812)]
[('own_rec', 0.19791666666666666)]
[('own_rec', 0.02555405888607942)]
5 1 1000



train precision@50, recall@50, precision@5, recall@5
[('own_rec', 0.09779166666666668)]
[('own_rec', 0.07465273837337628)]
[('own_rec', 0.2427777777777778)]
[('own_rec', 0.020079609443248887)]
test precision@50, recall@50, precision@5, recall@5
[('own_rec', 0.06697222222222222)]
[('own_rec', 0.0758743102477799)]
[('own_rec', 0.17930555555555558)]
[('own_rec', 0.02388773706313394)]
5 1 2000



train precision@50, recall@50, precision@5, recall@5
[('own_rec', 0.0954861111111111)]
[('own_rec', 0.07423664083246154)]
[('own_rec', 0.23194444444444445)]
[('own_rec', 0.019768229292575427)]
test precision@50, recall@50, precision@5, recall@5
[('own_rec', 0.0647361111111111)]
[('own_rec', 0.0741486557545851)]
[('own_rec', 0.16972222222222222)]
[('own_rec', 0.022691953283676222)]
5 1 3000



train precision@50, recall@50, precision@5, recall@5
[('own_rec', 0.0898888888888889)]
[('own_rec', 0.07093544828468451)]
[('own_rec', 0.21805555555555556)]
[('own_rec', 0.01851537809523881)]
test precision@50, recall@50, precision@5, recall@5
[('own_rec', 0.06154166666666667)]
[('own_rec', 0.07167587644092067)]
[('own_rec', 0.15916666666666668)]
[('own_rec', 0.02134971342252847)]
5 1 4000


...


train precision@50, recall@50, precision@5, recall@5
[('own_rec', 0.1557360406091371)]
[('own_rec', 0.06081168158661811)]
[('own_rec', 0.33313995649021033)]
[('own_rec', 0.015265292882549094)]
test precision@50, recall@50, precision@5, recall@5
[('own_rec', 0.05876722262509065)]
[('own_rec', 0.06608278077459871)]
[('own_rec', 0.14981870920957216)]
[('own_rec', 0.019687098815972623)]
12 6 3000



train precision@50, recall@50, precision@5, recall@5
[('own_rec', 0.14519216823785352)]
[('own_rec', 0.057670817215926685)]
[('own_rec', 0.3228426395939087)]
[('own_rec', 0.014945286333962488)]
test precision@50, recall@50, precision@5, recall@5
[('own_rec', 0.05422770123277738)]
[('own_rec', 0.06221274996895322)]
[('own_rec', 0.14010152284263958)]
[('own_rec', 0.019252081370449457)]
12 6 4000



train precision@50, recall@50, precision@5, recall@5
[('own_rec', 0.13411167512690358)]
[('own_rec', 0.0532564544797452)]
[('own_rec', 0.29804205946337925)]
[('own_rec', 0.013389365668619656)]
test precision@50, recall@50, precision@5, recall@5
[('own_rec', 0.04918056562726613)]
[('own_rec', 0.05661958390204053)]
[('own_rec', 0.13096446700507616)]
[('own_rec', 0.017970999109723156)]
12 6 5000



train precision@50, recall@50, precision@5, recall@5
[('own_rec', 0.12272661348803482)]
[('own_rec', 0.04894051623099041)]
[('own_rec', 0.27541696881798405)]
[('own_rec', 0.012068207353993296)]
test precision@50, recall@50, precision@5, recall@5
[('own_rec', 0.04485859318346628)]
[('own_rec', 0.051270305837320056)]
[('own_rec', 0.12139231327048586)]
[('own_rec', 0.016244324404093482)]
12 6 6000



train precision@50, recall@50, precision@5, recall@5
[('own_rec', 0.11183466279912982)]
[('own_rec', 0.04531164018614917)]
[('own_rec', 0.2555474981870921)]
[('own_rec', 0.011185180720237936)]
test precision@50, recall@50, precision@5, recall@5
[('own_rec', 0.04111675126903554)]
[('own_rec', 0.047362688294649166)]
[('own_rec', 0.11399564902102972)]
[('own_rec', 0.015095674605601392)]


In [ ]:
for VAL_MATCHER_WEEKS in matcher_weeks:
    for VAL_RANKER_WEEKS in ranker_weeks:
        for n_pop in n_pops:
            print(VAL_MATCHER_WEEKS, VAL_RANKER_WEEKS, n_pop)
            data_train_matcher, data_val_matcher, data_train_ranker, data_val_ranker = make_3_df(data, VAL_MATCHER_WEEKS, VAL_RANKER_WEEKS)
            data_train_matcher = prefilter_items(data_train_matcher, item_features=item_features, take_n_popular=n_pop)
            data_train_matcher, data_val_matcher, data_train_ranker, data_val_ranker, df_test = find_comm_us(data_train_matcher, data_val_matcher, data_train_ranker, data_val_ranker, df_test)
            recommender = MainRecommender(data_train_matcher)
            
            df_match_candidates = pd.DataFrame(data_train_ranker[USER_COL].unique())
            df_match_candidates.columns = [USER_COL]
            df_match_candidates['candidates'] = df_match_candidates[USER_COL].apply(lambda x: recommender.get_own_recommendations(x, N=N_PREDICT))
            names_rec = ['als_rec']
            list_models = [recommender.get_als_recommendations]

            make_df_recs(df_match_candidates, names_rec, list_models, N=N_PREDICT)
            df_match_candidates['candidates'] = df_match_candidates['user_id'].apply(lambda x: list(df_match_candidates[df_match_candidates['user_id']==x]['candidates'])[0]+
                                                                        list(df_match_candidates[df_match_candidates['user_id']==x]['als_rec'])[0]+
                                                                        list(list(result_eval_matcher[result_eval_matcher['user_id']==x]['actual'])[0])[:1])

            df_items = df_match_candidates.apply(lambda x: pd.Series(x['candidates']), axis=1).stack().reset_index(level=1, drop=True)
            df_items.name = 'item_id'
            df_match_candidates = df_match_candidates.drop(['candidates', 'als_rec'], axis=1).join(df_items)
            df_ranker_train = data_train_ranker[[USER_COL, ITEM_COL]].copy()
            df_ranker_train['target'] = 1  # тут только покупки
            df_ranker_train = df_match_candidates.merge(df_ranker_train, on=[USER_COL, ITEM_COL], how='left')
            df_ranker_train['target'].fillna(0, inplace= True)
            df_join_train_matcher = pd.concat([data_train_matcher, data_val_matcher])
            df_ranker_train = df_ranker_train.merge(item_features, on='item_id', how='left')
            df_ranker_train = df_ranker_train.merge(user_features, on='user_id', how='left')
            df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=ITEM_COL).agg('sales_value').sum().rename('total_item_sales_value'), how='left',on=ITEM_COL)
            df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=ITEM_COL).agg('quantity').sum().rename('total_quantity_value'), how='left',on=ITEM_COL)
            df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=ITEM_COL).agg(USER_COL).count().rename('item_freq'), how='left',on=ITEM_COL)
            df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=USER_COL).agg(USER_COL).count().rename('user_freq'), how='left',on=USER_COL)
            df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=USER_COL).agg('sales_value').sum().rename('total_user_sales_value'), how='left',on=USER_COL)
            df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=ITEM_COL).agg('quantity').sum().rename('item_quantity_per_week')/df_join_train_matcher.week_no.nunique(), how='left',on=ITEM_COL)
            df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=USER_COL).agg('quantity').sum().rename('user_quantity_per_week')/df_join_train_matcher.week_no.nunique(), how='left',on=USER_COL)
            df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=ITEM_COL).agg('quantity').sum().rename('item_quantity_per_basket')/df_join_train_matcher.basket_id.nunique(), how='left',on=ITEM_COL)
            df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=USER_COL).agg('quantity').sum().rename('user_quantity_per_baskter')/df_join_train_matcher.basket_id.nunique(), how='left',on=USER_COL)
            df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=ITEM_COL).agg(USER_COL).count().rename('item_freq_per_basket')/df_join_train_matcher.basket_id.nunique(), how='left',on=ITEM_COL)
            df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=USER_COL).agg(USER_COL).count().rename('user_freq_per_basket')/df_join_train_matcher.basket_id.nunique(), how='left',on=USER_COL)
            df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=USER_COL).agg('basket_id').nunique().rename('count_basket'), how='left',on=USER_COL)
            df_ranker_train['manufacturer'] = df_ranker_train['manufacturer'].astype(str)
            df_ranker_train['target'] = df_ranker_train['target'].astype(int)
            df_ranker_train['department'].fillna('nan', inplace= True)
            df_ranker_train['brand'].fillna('nan', inplace= True)
            df_ranker_train['commodity_desc'].fillna('nan', inplace= True)
            df_ranker_train['sub_commodity_desc'].fillna('nan', inplace= True)
            df_ranker_train['curr_size_of_product'].fillna('nan', inplace= True)
            df_ranker_train['age_desc'].fillna('nan', inplace= True)
            df_ranker_train['marital_status_code'].fillna('nan', inplace= True)
            df_ranker_train['income_desc'].fillna('nan', inplace= True)
            df_ranker_train['marital_status_code'].fillna('nan', inplace= True)
            df_ranker_train['income_desc'].fillna('nan', inplace= True)
            df_ranker_train['homeowner_desc'].fillna('nan', inplace= True)
            df_ranker_train['hh_comp_desc'].fillna('nan', inplace= True)
            df_ranker_train['household_size_desc'].fillna('nan', inplace= True)
            df_ranker_train['kid_category_desc'].fillna('nan', inplace= True)
            df_ranker_train['total_quantity_value'].fillna(0, inplace= True)
            df_ranker_train['item_freq'].fillna(0, inplace= True)
            df_ranker_train['user_freq'].fillna(0, inplace= True)
            df_ranker_train['count_basket'].fillna(0, inplace= True)
            X_train = df_ranker_train.drop('target', axis=1)
            y_train = df_ranker_train[['target']]
            cat_feats = ['manufacturer', 'department', 'brand', 'commodity_desc', 'sub_commodity_desc', 'curr_size_of_product',
                         'age_desc', 'marital_status_code', 'income_desc', 'homeowner_desc','hh_comp_desc','household_size_desc',
                         'kid_category_desc', 'total_quantity_value', 'item_freq', 'user_freq','count_basket']
            X_train[cat_feats] = X_train[cat_feats].astype('category')
            lgb = LGBMClassifier(objective='binary', max_depth=10, n_estimators=500,
                     learning_rate=0.05,
                     categorical_column=cat_feats)

            lgb.fit(X_train, y_train)
            train_preds = lgb.predict_proba(X_train)
            df_ranker_predict = df_ranker_train.copy()
            df_ranker_predict['proba_item_purchase'] = train_preds[:,1]
            df_ranker_predict = df_ranker_predict.drop_duplicates()
            df_ranker_predict.reset_index(drop=True, inplace = True)
            
            result_test = make_df_result(df_test)
            result_test['reranked_own_rec'] = result_test[USER_COL].apply(lambda user_id: rerank(user_id))
            print(sorted(calc_precision(result_test, 5), key=lambda x: x[1], reverse=True))




5 1 500



[LightGBM] [Warning] Accuracy may be bad since you didn't set num_leaves and 2^max_depth > num_leaves
[('reranked_own_rec', 0.23466279912980423)]
5 1 1000



[LightGBM] [Warning] Accuracy may be bad since you didn't set num_leaves and 2^max_depth > num_leaves
[('reranked_own_rec', 0.25380710659898476)]
5 1 2000



[LightGBM] [Warning] Accuracy may be bad since you didn't set num_leaves and 2^max_depth > num_leaves
[('reranked_own_rec', 0.2632342277012328)]
5 1 3000



[LightGBM] [Warning] Accuracy may be bad since you didn't set num_leaves and 2^max_depth > num_leaves
[('reranked_own_rec', 0.2658448150833938)]
5 1 4000


...


[LightGBM] [Warning] Accuracy may be bad since you didn't set num_leaves and 2^max_depth > num_leaves
[('reranked_own_rec', 0.23959390862944166)]
10 5 1000



[LightGBM] [Warning] Accuracy may be bad since you didn't set num_leaves and 2^max_depth > num_leaves
[('reranked_own_rec', 0.2548223350253807)]
10 5 2000



[LightGBM] [Warning] Accuracy may be bad since you didn't set num_leaves and 2^max_depth > num_leaves
[('reranked_own_rec', 0.25888324873096447)]
10 5 3000



[LightGBM] [Warning] Accuracy may be bad since you didn't set num_leaves and 2^max_depth > num_leaves
[('reranked_own_rec', 0.26033357505438726)]
10 5 4000



[LightGBM] [Warning] Accuracy may be bad since you didn't set num_leaves and 2^max_depth > num_leaves
[('reranked_own_rec', 0.264394488759971)]
10 5 5000
